In [2]:
# pip install sentence-transformers 

In [3]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer  # 한국어 처리 모델
from sklearn.metrics.pairwise import cosine_similarity  # 유사도 분석

# 데이터셋 로드

## (1) 원천 데이터 (.txt)

zip 파일 압축 해제

In [4]:
'''
import zipfile
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TS_01. KAKAO({i}).zip"
    # 추출할 위치
    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data"
    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)
'''

'\nimport zipfile\n# 추출할 zip 파일 경로\nfor i in range(1,5):\n    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TS_01. KAKAO({i}).zip"\n    # 추출할 위치\n    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data"\n    # zipfile 객체 생성\n    with zipfile.ZipFile(zip_file_path, \'r\') as zip_ref:\n        # 추출\n        zip_ref.extractall(extract_path)\n'

파일 불러오기

In [5]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/mental-health-chatbot/data/Training/data"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df = pd.concat(df_list, ignore_index= False)

## (2) 라벨링 데이터 (.json)

zip 파일 압축 해제

In [6]:
'''
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TL_01. KAKAO({i}).zip"

    # 추출할 위치
    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data_labeling"

    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)
'''

'\n# 추출할 zip 파일 경로\nfor i in range(1,5):\n    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TL_01. KAKAO({i}).zip"\n\n    # 추출할 위치\n    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data_labeling"\n\n    # zipfile 객체 생성\n    with zipfile.ZipFile(zip_file_path, \'r\') as zip_ref:\n        # 추출\n        zip_ref.extractall(extract_path)\n'

In [7]:
'''
import zipfile

# 추출할 zip 파일 경로
zip_file_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO.zip"

# 추출할 위치
extract_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO"

# zipfile 객체 생성
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:

    # 추출
    zip_ref.extractall(extract_path)
'''

'\nimport zipfile\n\n# 추출할 zip 파일 경로\nzip_file_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO.zip"\n\n# 추출할 위치\nextract_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO"\n\n# zipfile 객체 생성\nwith zipfile.ZipFile(zip_file_path, \'r\') as zip_ref:\n\n    # 추출\n    zip_ref.extractall(extract_path)\n'

파일 불러오기

In [8]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/mental-health-chatbot/data/Training/data_labeling"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df_json = pd.concat(df_list, ignore_index= False)

## (3) 데이터 확인

In [9]:
result_df  # 원천 데이터 

,0,구분
0,1 : 애덜앙 나 너무 배불러서 배 아파,1
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,3 : 얼마나 먹었다고 배가 아프데,1
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
11,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651
12,1 : 근데 해리만큼 잘 할 수 있을까?,71651
13,2 : 진짜 그 3인방 케미가 최고였는데,71651
14,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651


In [10]:
result_df_json  # 라벨링 데이터 

,0,구분
0,{,1
1,"""dataset"": {",1
2,"""identifier"": 71459,",1
3,"""name"": ""KAKAO_1000_01_set"",",1
4,"""src_path"": ""/data/file/cubeManager/PROJEC...",1
...,...,...
222,],71651
223,},71651
224,},71651
225,],71651


In [11]:
result_df_json[0][28]  # text

28            "text": "1 : 애덜앙 나 너무 배불러서 배 아파\n2 : 배...
28            "text": "1 : 아 얘들아 나 집들이 할 테니까 너네가 음식 ...
28            "text": "1 : 응원은 돈으로 받아.\n1 : 아 그리고 전세...
28            "text": "1 : 그냥 차 주는 회사 취업할게 내가. 면허는 누...
28            "text": "1 : 군인들은 산재 처리 되나?\n1 : 엄청 많이...
                            ...                        
28            "text": "1 : 나 머리 할까?\n1 : 뭔가 머리에 변화를 ...
28            "text": "1 : 너 사랑니 뽑았어?\n2 : 안 그래도 치과 ...
28            "text": "1 : 너가 저번 주에 택배 보냈었잖아\n2 : 응응...
28            "text": "1 : 요새 확실히 TV 안 보는 것 같아\n2 : ...
28            "text": "1 : 추석 잘 보냈어?\n2 : 응~ 영화 많이 해...
Name: 0, Length: 71651, dtype: object

In [12]:
result_df_json[0][6]  # 카테고리 (숫자)

6        "category": 2,
6        "category": 2,
6        "category": 2,
6        "category": 2,
6        "category": 2,
            ...        
6        "category": 2,
6        "category": 2,
6        "category": 2,
6        "category": 2,
6        "category": 2,
Name: 0, Length: 71651, dtype: object

In [13]:
result_df_json[0][20]  # 카테고리(문자)

20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
                ...            
20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
20          "category": "일상대화",
Name: 0, Length: 71651, dtype: object

In [14]:
result_df_json[0][24] # 주제 

24                 "subject": "식음료",
24              "subject": "주거와 생활",
24                  "subject": "교통",
24            "subject": "회사/아르바이트",
24                  "subject": "군대",
                   ...              
24                  "subject": "미용",
24                  "subject": "건강",
24              "subject": "상거래 전반",
24               "subject": "방송/연예",
24               "subject": "영화/만화",
Name: 0, Length: 71651, dtype: object

In [15]:
result_df_json[0][39] # 의도

39                   "speechAct": "(단언) 진술하기",
39                "speechAct": "(지시) 충고/제안하기",
39                   "speechAct": "(단언) 주장하기",
39                   "speechAct": "(단언) 주장하기",
39                   "speechAct": "(지시) 질문하기",
                        ...                   
39                   "speechAct": "(지시) 질문하기",
39                   "speechAct": "(지시) 질문하기",
39                   "speechAct": "(단언) 진술하기",
39                   "speechAct": "(단언) 진술하기",
39                   "speechAct": "(지시) 질문하기",
Name: 0, Length: 71651, dtype: object

# 전처리

json에서 주제, 의도, 화자수 컬럼 가져오기

In [45]:
# 주제
df_subject = result_df_json[0][24].to_frame()
df_subject

,0
24,"""subject"": ""식음료"","
24,"""subject"": ""주거와 생활"","
24,"""subject"": ""교통"","
24,"""subject"": ""회사/아르바이트"","
24,"""subject"": ""군대"","
...,...
24,"""subject"": ""미용"","
24,"""subject"": ""건강"","
24,"""subject"": ""상거래 전반"","
24,"""subject"": ""방송/연예"","


In [46]:
# 의도
df_sp = result_df_json[0][39]
sp = [df_sp.values]
sp

[array(['            "speechAct": "(단언) 진술하기",',
        '            "speechAct": "(지시) 충고/제안하기",',
        '            "speechAct": "(단언) 주장하기",', ...,
        '            "speechAct": "(단언) 진술하기",',
        '            "speechAct": "(단언) 진술하기",',
        '            "speechAct": "(지시) 질문하기",'], dtype=object)]

In [47]:
# 화자수
df_ss = result_df_json[0][25]
df_ss = df_ss.values
df_ss

array(['        "speaker_type": "다자간 대화",',
       '        "speaker_type": "다자간 대화",',
       '        "speaker_type": "다자간 대화",', ...,
       '        "speaker_type": "1:1",', '        "speaker_type": "1:1",',
       '        "speaker_type": "1:1",'], dtype=object)

In [48]:
indexs = range(1,71652,1)
len(indexs)

71651

In [49]:
# 주제, 의도, 화자수 컬럼 추가 

df_subject['구분'] = indexs
df_subject['의도'] = df_sp.values
df_subject['화자수'] = df_ss

df = pd.merge(result_df,df_subject,on='구분',how='left')
df = df.rename(columns={'0_x': '대화','0_y':'주제'})
df.head()

,대화,구분,주제,의도,화자수
0,1 : 애덜앙 나 너무 배불러서 배 아파,1,"""subject"": ""식음료"",","""speechAct"": ""(단언) 진술하기"",","""speaker_type"": ""다자간 대화"","
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1,"""subject"": ""식음료"",","""speechAct"": ""(단언) 진술하기"",","""speaker_type"": ""다자간 대화"","
2,3 : 얼마나 먹었다고 배가 아프데,1,"""subject"": ""식음료"",","""speechAct"": ""(단언) 진술하기"",","""speaker_type"": ""다자간 대화"","
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1,"""subject"": ""식음료"",","""speechAct"": ""(단언) 진술하기"",","""speaker_type"": ""다자간 대화"","
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1,"""subject"": ""식음료"",","""speechAct"": ""(단언) 진술하기"",","""speaker_type"": ""다자간 대화"","


문자열 정리

In [50]:
df['주제'] = df['주제'].str.split(':').str[1].str.strip()
df['주제'] = df['주제'].str[:-2].str[1:]
df['주제']

0            식음료
1            식음료
2            식음료
3            식음료
4            식음료
           ...  
1190458    영화/만화
1190459    영화/만화
1190460    영화/만화
1190461    영화/만화
1190462    영화/만화
Name: 주제, Length: 1190463, dtype: object

In [51]:
df['의도'] = df['의도'].str.split(':').str[1]
df['의도'] = df['의도'].str[:-2]
df['의도']=df['의도'].str[2:]
df['의도']

0          (단언) 진술하기
1          (단언) 진술하기
2          (단언) 진술하기
3          (단언) 진술하기
4          (단언) 진술하기
             ...    
1190458    (지시) 질문하기
1190459    (지시) 질문하기
1190460    (지시) 질문하기
1190461    (지시) 질문하기
1190462    (지시) 질문하기
Name: 의도, Length: 1190463, dtype: object

In [52]:
df['화자수'] = df['화자수'].str.split('_')
df['화자수'] = df['화자수'].str[1].str.split('"').str[2]
df['화자수']

0          다자간 대화
1          다자간 대화
2          다자간 대화
3          다자간 대화
4          다자간 대화
            ...  
1190458       1:1
1190459       1:1
1190460       1:1
1190461       1:1
1190462       1:1
Name: 화자수, Length: 1190463, dtype: object

In [53]:
df

,대화,구분,주제,의도,화자수
0,1 : 애덜앙 나 너무 배불러서 배 아파,1,식음료,(단언) 진술하기,다자간 대화
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1,식음료,(단언) 진술하기,다자간 대화
2,3 : 얼마나 먹었다고 배가 아프데,1,식음료,(단언) 진술하기,다자간 대화
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1,식음료,(단언) 진술하기,다자간 대화
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1,식음료,(단언) 진술하기,다자간 대화
...,...,...,...,...,...
1190458,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651,영화/만화,(지시) 질문하기,1:1
1190459,1 : 근데 해리만큼 잘 할 수 있을까?,71651,영화/만화,(지시) 질문하기,1:1
1190460,2 : 진짜 그 3인방 케미가 최고였는데,71651,영화/만화,(지시) 질문하기,1:1
1190461,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651,영화/만화,(지시) 질문하기,1:1


In [54]:
df = df[df['화자수']=='1:1']
df

,대화,구분,주제,의도,화자수
1060,1 : 배달 자주 시켜 먹는 편이야?,41,식음료,(지시) 질문하기,1:1
1061,2 : 아니 주말에는 배달 시켜 먹는 거 같아,41,식음료,(지시) 질문하기,1:1
1062,1 : 주말에만?,41,식음료,(지시) 질문하기,1:1
1063,1 : 그럼 평일에는 집밥 먹구?,41,식음료,(지시) 질문하기,1:1
1064,2 : 평일에는 거의 닭가슴살 ㅠㅠ,41,식음료,(지시) 질문하기,1:1
...,...,...,...,...,...
1190458,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651,영화/만화,(지시) 질문하기,1:1
1190459,1 : 근데 해리만큼 잘 할 수 있을까?,71651,영화/만화,(지시) 질문하기,1:1
1190460,2 : 진짜 그 3인방 케미가 최고였는데,71651,영화/만화,(지시) 질문하기,1:1
1190461,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651,영화/만화,(지시) 질문하기,1:1


반려동물이 주제인 대화

In [55]:
df_pet = df[df['주제']=='반려동물']
df_pet

,대화,구분,주제,의도,화자수
1216,1 : 이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1
1217,2 : 아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1
1218,1 : 키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1
1219,2 : 며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1
1220,1 : 어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1
...,...,...,...,...,...
1190277,1 : 그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1
1190278,1 : 한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1
1190279,2 : 좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1
1190280,1 : 알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1


화자 구분하기 (대화나누기 컬럼)

In [56]:
df_pet['대화나누기'] = df_pet['대화'].str.split(':').str[0]
df_pet['대화나누기'] = df_pet['대화나누기'].str.strip()
df_pet

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19388\2603979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pet['대화나누기'] = df_pet['대화'].str.split(':').str[0]
C:\Users\Playdata\AppData\Local\Temp\ipykernel_19388\2603979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pet['대화나누기'] = df_pet['대화나누기'].str.strip()


,대화,구분,주제,의도,화자수,대화나누기
1216,1 : 이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1217,2 : 아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1218,1 : 키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1219,2 : 며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1220,1 : 어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190277,1 : 그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,1 : 한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1
1190279,2 : 좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2
1190280,1 : 알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1,1


In [57]:
df_pet = df_pet[(df_pet['대화나누기']=='1') | (df_pet['대화나누기']=='2')]
df_pet

,대화,구분,주제,의도,화자수,대화나누기
1216,1 : 이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1217,2 : 아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1218,1 : 키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1219,2 : 며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1220,1 : 어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190277,1 : 그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,1 : 한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1
1190279,2 : 좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2
1190280,1 : 알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1,1


In [59]:
# df_pet.to_csv('df_pet.csv')

In [60]:
df_pet['대화'] = df_pet['대화'].str.split(':').str[1].str.strip()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19388\3042643659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pet['대화'] = df_pet['대화'].str.split(':').str[1].str.strip()


In [61]:
df_pet

,대화,구분,주제,의도,화자수,대화나누기
1216,이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1217,아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1218,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1219,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1220,어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190277,그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1
1190279,좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2
1190280,알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1,1


In [62]:
df_pet.to_csv('df_pet.csv')

user와 chatbot

In [63]:
df_pet_1 = df_pet[df_pet['대화나누기'] == '1']
df_pet_1

,대화,구분,주제,의도,화자수,대화나누기
1216,이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1218,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1220,어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
1222,거기 파충류 막 만지고 할 수 있어? 키키,49,반려동물,(지시) 질문하기,1:1,1
1224,오 키키 좋은 구경했겠다,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190273,그럼 어떻게 보라는 거야 키키,71640,반려동물,(지시) 질문하기,1:1,1
1190275,꾸릉이? 뭔가 미어캣이랑 연관성이 없는 느낌이야,71640,반려동물,(지시) 질문하기,1:1,1
1190277,그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1


In [64]:
df_pet_2 = df_pet[df_pet['대화나누기'] == '2']
df_pet_2

,대화,구분,주제,의도,화자수,대화나누기
1217,아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1219,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1221,제파라고 제주도에 있어 하하,49,반려동물,(지시) 질문하기,1:1,2
1223,아 키키 체험하는 곳이라서 만질 수 있다,49,반려동물,(지시) 질문하기,1:1,2
1226,거칠거칠한 느낌?,49,반려동물,(지시) 질문하기,1:1,2
...,...,...,...,...,...,...
1190271,근데 웃긴 건 유튜버 이름은 기억 안나,71640,반려동물,(지시) 질문하기,1:1,2
1190274,걱정 마 미어캣 이름은 알아 꾸릉이야,71640,반려동물,(지시) 질문하기,1:1,2
1190276,키키 근데 뭔가 잘 어울리지 않아?,71640,반려동물,(지시) 질문하기,1:1,2
1190279,좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2


In [65]:
df_pet

,대화,구분,주제,의도,화자수,대화나누기
1216,이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1217,아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1218,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1219,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1220,어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190277,그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1
1190279,좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2
1190280,알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1,1


In [66]:
df_pet.isnull().sum()

대화       0
구분       0
주제       0
의도       0
화자수      0
대화나누기    0
dtype: int64

화자 구분해서 csv 파일로 저장하기  
- 저장하면서 밀리게 되어서 NaN 값 발생하는 문제 발생

In [39]:
# df_pet_1.to_csv('df_pet_1.csv')

In [40]:
# df_pet_2.to_csv('df_pet_2.csv')

excel 파일로 저장하기

In [41]:
# df_pet_1.to_excel('df_pet_1.xlsx')

In [42]:
# df_pet_2.to_excel('df_pet_2.xlsx')

In [67]:
df_pet

,대화,구분,주제,의도,화자수,대화나누기
1216,이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1217,아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
1218,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
1219,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
1220,어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1
...,...,...,...,...,...,...
1190277,그런 것 같기도 하고?,71640,반려동물,(지시) 질문하기,1:1,1
1190278,한번 봐봐야지,71640,반려동물,(지시) 질문하기,1:1,1
1190279,좋아좋아 꾸릉이라고 검색하고 한번 봐봐,71640,반려동물,(지시) 질문하기,1:1,2
1190280,알겠어 내가 보고 후기 알려줄게 키키,71640,반려동물,(지시) 질문하기,1:1,1


In [74]:
df_pet_test['대화나누기'].astype('int')

1216       1
1217       2
1218       1
1219       2
1220       1
          ..
1190277    1
1190278    1
1190279    2
1190280    1
1190281    2
Name: 대화나누기, Length: 37209, dtype: int32

In [80]:
df_pet_test = df_pet.copy()

In [89]:
lst = []
lst.append(df_pet_test[['대화','대화나누기']].values[0])
lst

[array(['이구아나나 파충류 키워본 적 있나?', '1'], dtype=object)]

In [90]:
from tqdm import tqdm
for i in tqdm(range(1, len(df_pet_test))) :
    if (df_pet_test['대화나누기'].values[i]) != (lst[-1][1]):
        lst.append(df_pet_test[['대화','대화나누기']].values[i])
    else:
        lst[-1][0] += df_pet_test[['대화','대화나누기']].values[i][0]
print(lst)

100%|██████████| 37208/37208 [00:50<00:00, 743.04it/s]


[array(['이구아나나 파충류 키워본 적 있나?', '1'], dtype=object), array(['아니 무서워서 어떻게 키워…', '2'], dtype=object), array(['키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ', '1'], dtype=object), array(['며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서', '2'], dtype=object), array(['어디 동물원 갔었엉?', '1'], dtype=object), array(['제파라고 제주도에 있어 하하', '2'], dtype=object), array(['거기 파충류 막 만지고 할 수 있어? 키키', '1'], dtype=object), array(['아 키키 체험하는 곳이라서 만질 수 있다', '2'], dtype=object), array(['오 키키 좋은 구경했겠다이구아나 만졌을 때 느낌 어땠어?', '1'], dtype=object), array(['거칠거칠한 느낌?무서웠다', '2'], dtype=object), array(['어우 키키 말만 들어도 무섭농', '1'], dtype=object), array(['너는 이구아나 본 적 있어?', '2'], dtype=object), array(['아니 ㅠㅠ 나는 한번도 없어... 키키', '1'], dtype=object), array(['키키 나도 여친 아니었으면 볼 일 없었을 거 같아', '2'], dtype=object), array(['여친이 제파? 가자고 했던 거야?', '1'], dtype=object), array(['어 키키 가보고 싶다고 해서 갔지', '2'], dtype=object), array(['수성못 갔을 때 수달 봤었는데 너무 귀엽더라 ㅋ', '1'], dtype=object), array(['수성못에 수달도 있나?', '2'], dtype=object), array(['응 키키 한번 봤었다 물에서 나왔다가 쏙 들어가더랑', '1'], dty

In [95]:
type(lst)

list

In [98]:
df_lst = pd.DataFrame(lst)

In [99]:
df_lst

,0,1
0,이구아나나 파충류 키워본 적 있나?,1
1,아니 무서워서 어떻게 키워…,2
2,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,1
3,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,2
4,어디 동물원 갔었엉?,1
...,...,...
31905,키키 근데 뭔가 잘 어울리지 않아?,2
31906,그런 것 같기도 하고?한번 봐봐야지,1
31907,좋아좋아 꾸릉이라고 검색하고 한번 봐봐,2
31908,알겠어 내가 보고 후기 알려줄게 키키,1


In [ ]:
df_lst['0'].rename 